음주: 20 × (100/101) ≈ 19.80  
흡연: 38 × (100/101) ≈ 37.62  
걷기: 28 × (100/101) ≈ 27.72  
아침식사: 15 × (100/101) ≈ 14.85

**군집화의 피드백에 대한 실천율 벡터화**

In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from math import pi
import matplotlib.pyplot as plt
import uuid

# 시각화 설정 (한글 폰트 및 음수 표시 설정)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 로드 (실제 파일 경로에 맞게 수정해주세요)
df = pd.read_csv("건강데이터_2022_2023_합본.csv")
df = df[(df['HE_glu'] >= 50) & (df['HE_glu'] <= 400)]

# 전처리: BE3_31 (1주일 간 걷기 일수)
df['BE3_31'] = df['BE3_31'].astype(str).str.strip().replace({
    '1.0': 0, '2.0': 1, '3.0': 2, '4.0': 3, '5.0': 4, '6.0': 5,
    '7.0': 6, '8.0': 7, '88.0': 0, '99.0': np.nan, 'nan': np.nan
})
df['BE3_31'] = pd.to_numeric(df['BE3_31'], errors='coerce')

# 전처리: L_BR_FQ (최근 1년 동안 1주 동안 아침식사 빈도)
df['L_BR_FQ'] = df['L_BR_FQ'].astype(str).str.strip().replace({
    '1.0': 6, '2.0': 3.5, '3.0': 1.5, '4.0': 0, '9.0': np.nan, 'nan': np.nan
})
df['L_BR_FQ'] = pd.to_numeric(df['L_BR_FQ'], errors='coerce')

# 전처리: tobacco (일반 담배 + 전자 담배 하루 평균 흡연량)
df['BS3_2'] = df['BS3_2'].replace(888, 0)
df['BS12_47_1'] = df['BS12_47_1'].replace({888: 0, 999: np.nan})
df['BS3_2'] = pd.to_numeric(df['BS3_2'], errors='coerce')
df['BS12_47_1'] = pd.to_numeric(df['BS12_47_1'], errors='coerce')
df['tobacco'] = df[['BS3_2', 'BS12_47_1']].sum(axis=1, skipna=True)
df['tobacco'] = df['tobacco'].astype(str).str.strip().replace({'999.0': np.nan})
df['tobacco'] = pd.to_numeric(df['tobacco'], errors='coerce')

# 전처리: BD1_11 (1년간 음주 빈도) - 연간 횟수로 변환
df['BD1_11'] = df['BD1_11'].astype(str).str.strip().replace({
    '1.0': 0, '2.0': 6, '3.0': 12, '4.0': 42, '5.0': 130, '6.0': 208,
    '8.0': 0, '9.0': np.nan, 'nan': np.nan
})
df['BD1_11'] = pd.to_numeric(df['BD1_11'], errors='coerce')

# 고혈압 파생변수 생성 (HE_HP2)
def determine_he_hp2(sbp, dbp):
    if pd.isna(sbp) or pd.isna(dbp):
        return np.nan
    if sbp >= 140 or dbp >= 90:
        return 4
    elif 130 <= sbp <= 139 or 80 <= dbp <= 89:
        return 3
    elif 120 <= sbp <= 129 and dbp < 80:
        return 2
    else:
        return 1

# 당뇨 파생변수 생성 (HE_DM_HbA1c2)
def determine_he_dm_hba1c2(glu):
    if pd.isna(glu):
        return np.nan
    if glu <= 99:
        return 1
    elif 100 <= glu <= 125:
        return 2
    else:
        return 3

# 비만 파생변수 생성 (HE_obe2)
def determine_he_obe2(bmi):
    if pd.isna(bmi):
        return np.nan
    if bmi <= 18.5:
        return 1
    elif 18.5 < bmi <= 22.9:
        return 2
    elif 22.9 < bmi <= 24.9:
        return 3
    elif 24.9 < bmi <= 29.9:
        return 4
    elif 29.9 < bmi <= 34.9:
        return 5
    else:
        return 6

df['HE_HP2'] = df.apply(lambda row: determine_he_hp2(row['HE_sbp1'], row['HE_dbp1']), axis=1)
df['HE_DM_HbA1c2'] = df.apply(lambda row: determine_he_dm_hba1c2(row['HE_glu']), axis=1)
df['HE_obe2'] = df.apply(lambda row: determine_he_obe2(row['HE_BMI']), axis=1)
df['HE_HP2'] = pd.to_numeric(df['HE_HP2'], errors='coerce')
df['HE_DM_HbA1c2'] = pd.to_numeric(df['HE_DM_HbA1c2'], errors='coerce')
df['HE_obe2'] = pd.to_numeric(df['HE_obe2'], errors='coerce')

# 결측치 제거 및 군집 분석에 사용할 컬럼 선택
df_clustering = df.dropna(subset=['HE_HP2', 'HE_DM_HbA1c2', 'HE_obe2'])
selected_cols = ['BD1_11', 'tobacco', 'BE3_31', 'L_BR_FQ']

# 선택된 컬럼의 결측치를 최빈값으로 대체
for col in selected_cols:
    if df_clustering[col].isnull().any():
        mode_value = df_clustering[col].mode(dropna=True)
        if not mode_value.empty:
            df_clustering.loc[:, col] = df_clustering[col].fillna(mode_value[0])
        else:
            df_clustering.loc[:, col] = df_clustering[col].fillna(0)

# 사전 계산된 군집 평균 데이터
precalculated_means = {
    'HE_HP2': {
        1: pd.DataFrame({
            '1주일 간 음주 빈도': [0.212965, 0.346154, 0.263829, 0.283081, 3.773973, 0.532374, 3.081522],
            '하루 평균 흡연량': [0.180919, 0.803408, 0.243425, 0.696676, 18.815068, 18.654676, 1.434783],
            '1주일 간 걷기 일수': [1.006360, 5.964613, 6.167879, 1.013850, 3.102740, 3.805755, 4.766304],
            '1주일 간 아침식사 빈도': [5.547703, 0.613368, 5.585898, 0.623269, 2.616438, 3.769784, 3.337862]
        }, index=[0, 1, 2, 3, 4, 5, 6]),
        2: pd.DataFrame({
            '1주일 간 음주 빈도': [3.257282, 0.600634, 0.173377, 0.227414, 0.761134, 5.500000, 1.003752],
            '하루 평균 흡연량': [0.582524, 0.444444, 0.102253, 0.382550, 23.385965, 6.716981, 13.719512],
            '1주일 간 걷기 일수': [5.291262, 3.584229, 6.287695, 1.201342, 1.666667, 2.660377, 6.024390],
            '1주일 간 아침식사 빈도': [5.432039, 0.605735, 5.757366, 5.692394, 3.789474, 3.924528, 3.152439]
        }, index=[0, 1, 2, 3, 4, 5, 6]),
        3: pd.DataFrame({
            '1주일 간 음주 빈도': [0.517830, 0.747283, 2.083208, 0.473159, 5.500000, 1.960664],
            '하루 평균 흡연량': [0.255906, 1.091078, 19.352941, 0.420361, 2.595745, 17.242424],
            '1주일 간 걷기 일수': [6.213583, 3.780669, 1.895425, 1.308703, 4.085106, 5.469697],
            '1주일 간 아침식사 빈도': [5.778543, 0.697026, 2.271242, 5.749589, 4.049645, 5.295455]
        }, index=[0, 1, 2, 3, 4, 5]),
        4: pd.DataFrame({
            '1주일 간 음주 빈도': [5.500000, 0.463656, 0.471816, 2.740812, 0.930769],
            '하루 평균 흡연량': [2.284672, 0.332907, 0.317814, 20.272222, 1.310000],
            '1주일 간 걷기 일수': [3.729927, 6.289373, 1.212551, 3.850000, 3.813333],
            '1주일 간 아침식사 빈도': [4.463504, 5.839949, 5.731781, 3.738889, 0.693333]
        }, index=[0, 1, 2, 3, 4]),
    },
    'HE_DM_HbA1c2': {
        1: pd.DataFrame({
            '1주일 간 음주 빈도': [0.679037, 3.699045, 0.205307, 0.495192, 1.637577, 0.247029],
            '하루 평균 흡연량': [0.883784, 1.920382, 0.305901, 0.764344, 19.815618, 0.317173],
            '1주일 간 걷기 일수': [6.015315, 4.474522, 1.086957, 1.079918, 3.561822, 6.187346],
            '1주일 간 아침식사 빈도': [0.591892, 4.639331, 5.596273, 0.627049, 3.206074, 5.622021]
        }, index=[0, 1, 2, 3, 4, 5]),
        2: pd.DataFrame({
            '1주일 간 음주 빈도': [0.467910, 0.868637, 1.435988, 5.500000, 0.474998],
            '하루 평균 흡연량': [0.438253, 1.461412, 19.321555, 5.978166, 0.407047],
            '1주일 간 걷기 일수': [6.262048, 3.768473, 3.653710, 3.890830, 1.222357],
            '1주일 간 아침식사 빈도': [5.800452, 0.623974, 3.793286, 4.109170, 5.717497]
        }, index=[0, 1, 2, 3, 4]),
        3: pd.DataFrame({
            '1주일 간 음주 빈도': [0.365672, 0.374903, 1.797498, 0.454709, 2.443396, 4.320225],
            '하루 평균 흡연량': [0.492537, 0.199495, 21.493976, 0.953271, 19.056604, 1.000000],
            '1주일 간 걷기 일수': [1.223881, 6.353535, 3.795181, 3.943925, 3.622642, 3.662921],
            '1주일 간 아침식사 빈도': [5.763682, 5.886364, 5.879518, 0.668224, 1.179245, 5.382022]
        }, index=[0, 1, 2, 3, 4, 5]),
    },
    'HE_obe2': {
        1: pd.DataFrame({
            '1주일 간 음주 빈도': [3.462264, 0.056121, 0.347728, 0.147807, 1.490074, 0.463646],
            '하루 평균 흡연량': [3.226415, 0.073930, 0.485380, 0.297521, 19.774194, 0.698630],
            '1주일 간 걷기 일수': [3.094340, 0.443580, 6.187135, 0.719008, 3.967742, 5.945205],
            '1주일 간 아침식사 빈도': [2.943396, 5.659533, 5.576023, 0.619835, 3.064516, 0.534247]
        }, index=[0, 1, 2, 3, 4, 5]),
        2: pd.DataFrame({
            '1주일 간 음주 빈도': [1.346814, 0.619275, 0.418860, 5.237327, 0.692170, 0.278670],
            '하루 평균 흡연량': [18.386266, 0.748092, 0.245912, 7.737327, 0.670251, 0.306064],
            '1주일 간 걷기 일수': [3.497854, 1.219466, 6.217444, 3.718894, 6.069892, 1.216554],
            '1주일 간 아침식사 빈도': [3.343348, 0.885496, 5.662326, 4.158986, 0.585125, 5.677575]
        }, index=[0, 1, 2, 3, 4, 5]),
        3: pd.DataFrame({
            '1주일 간 음주 빈도': [0.428221, 0.465444, 1.405400, 0.733680, 5.500000],
            '하루 평균 흡연량': [0.353503, 0.406190, 19.929293, 0.981982, 5.000000],
            '1주일 간 걷기 일수': [1.168790, 6.233075, 3.691919, 3.645045, 4.143836],
            '1주일 간 아침식사 빈도': [5.685510, 5.787234, 3.845960, 0.740541, 4.191781]
        }, index=[0, 1, 2, 3, 4]),
        4: pd.DataFrame({
            '1주일 간 음주 빈도': [1.745897, 0.476494, 0.474684, 4.074257, 0.400403],
            '하루 평균 흡연량': [20.120000, 0.413905, 0.444304, 2.267327, 0.994178],
            '1주일 간 걷기 일수': [3.673333, 6.226354, 1.289873, 4.603960, 3.608443],
            '1주일 간 아침식사 빈도': [3.465000, 5.767583, 5.724684, 2.785479, 0.788937]
        }, index=[0, 1, 2, 3, 4]),
        5: pd.DataFrame({
            '1주일 간 음주 빈도': [0.313474, 0.188658, 1.018219, 4.313953, 0.697115, 2.906780, 0.266827],
            '하루 평균 흡연량': [0.215686, 0.218978, 17.921053, 3.069767, 23.687500, 2.000000, 1.331250],
            '1주일 간 걷기 일수': [6.202614, 1.160584, 3.368421, 2.488372, 3.781250, 4.915254, 3.893750],
            '1주일 간 아침식사 빈도': [5.722222, 5.635036, 5.605263, 5.174419, 0.984375, 0.838983, 0.659375]
        }, index=[0, 1, 2, 3, 4, 5, 6]),
        6: pd.DataFrame({
            '1주일 간 음주 빈도': [0.202830, 5.500000, 0.407692, 0.332308, 1.230769],
            '하루 평균 흡연량': [0.198113, 7.692308, 0.642857, 0.400000, 20.360000],
            '1주일 간 걷기 일수': [0.424528, 5.230769, 3.942857, 6.480000, 3.840000],
            '1주일 간 아침식사 빈도': [5.787736, 4.730769, 0.600000, 5.625000, 3.480000]
        }, index=[0, 1, 2, 3, 4])
    }
}

def get_user_input():
    name = "홍길동"
    sbp = 142.0  # 수축기 혈압
    dbp = 92.0   # 이완기 혈압
    glu = 128.0  # 공복혈당
    bmi = 27.5   # BMI
    weekly_alcohol = 3.0  # 1주일 간 음주 빈도 (회)
    daily_smoking = 10.0  # 하루 평균 흡연량 (개비)
    weekly_exercise = 2.0  # 1주일 간 걷기 일수 (일)
    daily_breakfast = 2.0  # 1주 동안 아침식사 빈도 (회)
    return name, sbp, dbp, glu, bmi, weekly_alcohol, daily_smoking, weekly_exercise, daily_breakfast

# 군집 결정 함수
def determine_cluster_with_precalculated_means(user_data_weekly_daily, condition_value, scaler, precalculated_means_dict, selected_cols, condition_type):
    condition_value_int = int(condition_value)
    if condition_type not in precalculated_means_dict or condition_value_int not in precalculated_means_dict[condition_type]:
        print(f"사전 계산된 군집 평균 데이터에 {condition_type}={condition_value_int} 그룹이 없습니다.")
        return None, None
    cluster_means_precalculated_text_names = precalculated_means_dict[condition_type][condition_value_int].copy()
    text_to_internal_col_mapping = {
        '1주일 간 음주 빈도': 'BD1_11',
        '하루 평균 흡연량': 'tobacco',
        '1주일 간 걷기 일수': 'BE3_31',
        '1주일 간 아침식사 빈도': 'L_BR_FQ'
    }
    cluster_means_precalculated_internal_names = pd.DataFrame(index=cluster_means_precalculated_text_names.index)
    for internal_col in selected_cols:
        text_col = next((k for k, v in text_to_internal_col_mapping.items() if v == internal_col), None)
        if text_col and text_col in cluster_means_precalculated_text_names.columns:
            cluster_means_precalculated_internal_names[internal_col] = cluster_means_precalculated_text_names[text_col]
        else:
            print(f"경고: 내부 컬럼 '{internal_col}'에 대한 텍스트 이름 매핑이 없거나 사전 계산된 데이터에 없습니다.")
            cluster_means_precalculated_internal_names[internal_col] = np.nan
    if cluster_means_precalculated_internal_names.isnull().values.any():
        print("경고: 스케일링하려는 사전 계산된 평균 데이터에 결측치가 포함되어 있습니다.")
        return None, None
    scaled_precalculated_centroids = scaler.transform(cluster_means_precalculated_internal_names[selected_cols])
    user_data_transformed_for_predict = [
        user_data_weekly_daily[0],  # 주간 음주
        user_data_weekly_daily[1],  # 하루 흡연
        user_data_weekly_daily[2],  # 주간 걷기
        user_data_weekly_daily[3]   # 아침식사
    ]
    user_data_scaled_for_predict = scaler.transform([user_data_transformed_for_predict])
    distances = np.linalg.norm(scaled_precalculated_centroids - user_data_scaled_for_predict, axis=1)
    closest_cluster = np.argmin(distances)
    cluster_means = precalculated_means_dict[condition_type][condition_value_int].iloc[closest_cluster]
    return closest_cluster, cluster_means

# 레이더 차트 생성 함수
def plot_radar_chart(user_data_weekly_daily, compare_cluster_mean_weekly_daily, categories, title):
    values_user = user_data_weekly_daily[:]
    values_cluster = compare_cluster_mean_weekly_daily[:]
    all_values = values_user + values_cluster
    max_val = max(all_values) if all_values else 1
    values_user = values_user + values_user[:1]
    values_cluster = values_cluster + values_cluster[:1]
    angles = [n / float(len(categories)) * 2 * pi for n in range(len(categories))]
    angles += angles[:1]
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(polar=True))
    ax.set_ylim(0, max_val * 1.2)
    ax.fill(angles, values_user, color='red', alpha=0.25, label='사용자')
    ax.plot(angles, values_user, color='red', linewidth=2)
    ax.fill(angles, values_cluster, color='blue', alpha=0.25, label='비교 군집 평균')
    ax.plot(angles, values_cluster, color='blue', linewidth=2)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize=10)
    ax.set_title(title, size=15, color='black', y=1.1)
    ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))
    ax.grid(True)
    plt.savefig(f"{title.replace(' ', '_')}.png")  # 이미지 저장
    plt.close()

# 건강 지표 계산 함수
def calculate_health_score(cluster_mean, condition_type):
    weights = {
        'HE_HP2': {
            'tobacco': 0.383886,
            'BE3_31': 0.277795,
            'BD1_11': 0.188051,
            'L_BR_FQ': 0.150268
        },
        'HE_DM_HbA1c2': {
            'tobacco': 0.362728,
            'BE3_31': 0.275424,
            'BD1_11': 0.199061,
            'L_BR_FQ': 0.162787
        },
        'HE_obe2': {
            'tobacco': 0.398547,
            'BE3_31': 0.265374,
            'BD1_11': 0.198054,
            'L_BR_FQ': 0.138025
        }
    }
    if condition_type not in weights:
        raise ValueError(f"알 수 없는 condition_type: {condition_type}")
    w = weights[condition_type]
    score = (
        -cluster_mean['1주일 간 음주 빈도'] * w['BD1_11']
        -cluster_mean['하루 평균 흡연량'] * w['tobacco']
        +cluster_mean['1주일 간 걷기 일수'] * w['BE3_31']
        +cluster_mean['1주일 간 아침식사 빈도'] * w['L_BR_FQ']
    )
    return score

# 코사인 유사도 계산 함수
def calculate_cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1 == 0 or norm_vec2 == 0:
        return 0
    return dot_product / (norm_vec1 * norm_vec2)

# 더 건강하고 유사한 군집 선택 함수
def find_healthier_and_similar_cluster(user_data_raw, user_cluster_means, condition_value, precalculated_means_dict, condition_type):
    condition_value_int = int(condition_value)
    if condition_type not in precalculated_means_dict or condition_value_int not in precalculated_means_dict[condition_type]:
        return None, None
    cluster_means_all = precalculated_means_dict[condition_type][condition_value_int]
    health_scores = cluster_means_all.apply(lambda row: calculate_health_score(row, condition_type), axis=1)
    user_cluster_score = calculate_health_score(user_cluster_means, condition_type)
    user_data_vector_raw = np.array([
        user_data_raw[0], user_data_raw[1], user_data_raw[2], user_data_raw[3]
    ])
    healthier_and_similar_cluster_index = None
    max_cosine_similarity = -1
    if user_cluster_score >= health_scores.max():
        return 'self', user_cluster_means
    for idx in cluster_means_all.index:
        current_cluster_mean = cluster_means_all.loc[idx]
        current_score = calculate_health_score(current_cluster_mean, condition_type)
        if current_score > user_cluster_score:
            cluster_mean_vector_raw = current_cluster_mean[[
                '1주일 간 음주 빈도', '하루 평균 흡연량', '1주일 간 걷기 일수', '1주일 간 아침식사 빈도'
            ]].values
            sim = calculate_cosine_similarity(user_data_vector_raw, cluster_mean_vector_raw)
            if sim > max_cosine_similarity:
                max_cosine_similarity = sim
                healthier_and_similar_cluster_index = idx
    if healthier_and_similar_cluster_index is not None:
        return healthier_and_similar_cluster_index, cluster_means_all.loc[healthier_and_similar_cluster_index]
    return None, None

# 체크리스트 클래스
class HealthChecklist:
    def __init__(self):
        self.feedback_items = []
        self.check_status = {}
        self.scores = {}
        self.total_score = 0
        self.max_score = 100
        # KNHANES 가중치를 반영한 점수 (101점 스케일링)
        base_scores = {
            '1주일 간 음주 빈도': 20,
            '하루 평균 흡연량': 38,
            '1주일 간 걷기 일수': 28,
            '1주일 간 아침식사 빈도': 15
        }
        self.item_scores = {
            key: round(score * (100/101), 2) for key, score in base_scores.items()
        }  # 스케일링: 음주 19.80, 흡연 37.62, 걷기 27.72, 아침식사 14.85

    def add_feedback(self, condition, item, action, value):
        feedback = {
            'condition': condition,
            'item': item,
            'action': action,
            'value': value
        }
        self.feedback_items.append(feedback)
        key = f"{condition}_{item}"
        self.check_status[key] = False
        self.scores[key] = 0

    def check_item(self, condition, item):
        key = f"{condition}_{item}"
        if key in self.check_status:
            self.check_status[key] = True
            self.scores[key] = self.item_scores[item]
            self._update_total_score()
        else:
            print(f"항목 {key}가 존재하지 않습니다.")

    def uncheck_item(self, condition, item):
        key = f"{condition}_{item}"
        if key in self.check_status:
            self.check_status[key] = False
            self.scores[key] = 0
            self._update_total_score()
        else:
            print(f"항목 {key}가 존재하지 않습니다.")

    def _update_total_score(self):
        self.total_score = round(sum(self.scores.values()), 2)

    def get_checklist_status(self):
        hp_score = round(sum(self.scores[f"고혈압_{item}"] for item in self.item_scores.keys() if f"고혈압_{item}" in self.scores), 2)
        dm_score = round(sum(self.scores[f"당뇨_{item}"] for item in self.item_scores.keys() if f"당뇨_{item}" in self.scores), 2)
        ob_score = round(sum(self.scores[f"비만_{item}"] for item in self.item_scores.keys() if f"비만_{item}" in self.scores), 2)
        total_completion_rate = round((hp_score + dm_score + ob_score) / 3, 2) if self.max_score > 0 else 0
        checklist_data = {
            'feedback_items': self.feedback_items,
            'check_status': self.check_status,
            'scores': {k: round(v, 2) for k, v in self.scores.items()},
            'total_score': self.total_score,
            'max_score': self.max_score,
            'completion_rate': total_completion_rate
        }
        return checklist_data

    def save_checklist(self, filename):
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.get_checklist_status(), f, ensure_ascii=False, indent=4)
        print(f"체크리스트가 {filename}에 저장되었습니다.")

    def display_checklist(self):
        print("\n=== 건강 체크리스트 ===")
        df_data = []
        for feedback in self.feedback_items:
            key = f"{feedback['condition']}_{feedback['item']}"
            df_data.append({
                '질병': feedback['condition'],
                '항목': feedback['item'],
                '액션': feedback['action'],
                '목표값': f"{feedback['value']:.2f}" if feedback['value'] != '적정' else '적정',
                '체크여부': '☑' if self.check_status[key] else '☐',
                '점수': round(self.scores[key], 2)
            })
        checklist_df = pd.DataFrame(df_data)
        print(checklist_df.to_markdown(index=False, floatfmt=".2f"))
        hp_score = round(sum(self.scores[f"고혈압_{item}"] for item in self.item_scores.keys() if f"고혈압_{item}" in self.scores), 2)
        dm_score = round(sum(self.scores[f"당뇨_{item}"] for item in self.item_scores.keys() if f"당뇨_{item}" in self.scores), 2)
        ob_score = round(sum(self.scores[f"비만_{item}"] for item in self.item_scores.keys() if f"비만_{item}" in self.scores), 2)
        total_completion_rate = round((hp_score + dm_score + ob_score) / 3, 2)
        print(f"\n고혈압 점수: {hp_score:.2f}/{self.max_score}")
        print(f"당뇨 점수: {dm_score:.2f}/{self.max_score}")
        print(f"비만 점수: {ob_score:.2f}/{self.max_score}")
        print(f"총 실천율: {total_completion_rate:.2f}%")

# 메인 서비스 함수
def integrated_health_service():
    name, sbp, dbp, glu, bmi, weekly_alcohol, daily_smoking, weekly_exercise, daily_breakfast = get_user_input()
    checklist = HealthChecklist()

    def analyze_condition(condition_type, condition_value, condition_labels, user_data):
        condition_value_int = int(condition_value)
        if condition_type not in precalculated_means or condition_value_int not in precalculated_means[condition_type]:
            print(f"\n{condition_type} 분류 {condition_labels[condition_value_int]}에 대한 사전 계산된 군집 평균 데이터가 없습니다.")
            return
        df_condition = df_clustering[df_clustering[condition_type] == condition_value_int][selected_cols].copy()
        if df_condition.empty:
            print(f"\n{condition_type} 분류 {condition_labels[condition_value_int]}에 해당하는 데이터가 부족하여 비교를 수행할 수 없습니다.")
            return
        scaler = StandardScaler()
        scaler.fit(df_condition[selected_cols])
        cluster, cluster_means = determine_cluster_with_precalculated_means(
            user_data, condition_value_int, scaler, precalculated_means, selected_cols, condition_type
        )
        if cluster is None:
            print(f"\n{condition_type} 분류 {condition_labels[condition_value_int]}에 대한 군집을 찾을 수 없습니다.")
            return
        target_cluster_index, target_cluster_means = find_healthier_and_similar_cluster(
            user_data, cluster_means, condition_value_int, precalculated_means, condition_type
        )
        comparison_title_suffix = f" ({condition_labels[condition_value_int]})"
        if target_cluster_index is not None and target_cluster_index != 'self':
            print(f"\n---\n## ✅ {condition_labels[condition_value_int]} 건강 목표 - 더 건강한 그룹과 나 비교하기")
            compare_values = target_cluster_means.tolist()
        else:
            print(f"\n현재 **{condition_labels[condition_value_int]}** 그룹에서 **{name}**님의 군집이 가장 건강하거나, 더 나은 군집이 없습니다. 현재 군집을 기준으로 건강 목표를 설정합니다.")
            compare_values = cluster_means.tolist()
        categories = ['1주일 간 음주 빈도', '하루 평균 흡연량', '1주일 간 걷기 일수', '1주일 간 아침식사 빈도']
        plot_radar_chart(user_data, compare_values, categories,
                         f"{name}님의 생활습관 vs 비교 군집 평균{comparison_title_suffix}")
        print("\n---\n## ✅ 사용자 생활습관 vs 비교 군집 평균 상세:")
        table_data = {
            '항목': categories,
            f'{name}님 수치': user_data,
            '비교 군집 수치': compare_values
        }
        comparison_df = pd.DataFrame(table_data)
        print(comparison_df.to_markdown(index=False, floatfmt=".2f"))
        print(f"\n**{name}**님은 **{condition_labels[condition_value_int]}**에 속합니다.")
        # 피드백 추가
        diff_alcohol = round(user_data[0] - target_cluster_means['1주일 간 음주 빈도'], 2)
        condition_name = {'HE_HP2': '고혈압', 'HE_DM_HbA1c2': '당뇨', 'HE_obe2': '비만'}[condition_type]
        if diff_alcohol > 0:
            checklist.add_feedback(condition_name, "1주일 간 음주 빈도", "줄여야", abs(diff_alcohol))
            print(f"1주일 간 음주 빈도는 **{abs(diff_alcohol)}회 줄여야** 합니다.")
        elif diff_alcohol < 0:
            checklist.add_feedback(condition_name, "1주일 간 음주 빈도", "낮습니다", abs(diff_alcohol))
            print(f"1주일 간 음주 빈도는 **{abs(diff_alcohol)}회 낮습니다.**")
        else:
            checklist.add_feedback(condition_name, "1주일 간 음주 빈도", "적정", "적정")
            print("1주일 간 음주 빈도는 **적정입니다.**")
        diff_smoking = round(user_data[1] - target_cluster_means['하루 평균 흡연량'], 2)
        if diff_smoking > 0:
            checklist.add_feedback(condition_name, "하루 평균 흡연량", "줄여야", abs(diff_smoking))
            print(f"하루 평균 흡연량은 **{abs(diff_smoking)}개비 줄여야** 합니다.")
        elif diff_smoking < 0:
            checklist.add_feedback(condition_name, "하루 평균 흡연량", "낮습니다", abs(diff_smoking))
            print(f"하루 평균 흡연량은 **{abs(diff_smoking)}개비 낮습니다.**")
        else:
            checklist.add_feedback(condition_name, "하루 평균 흡연량", "적정", "적정")
            print("하루 평균 흡연량은 **적정입니다.**")
        diff_exercise = round(target_cluster_means['1주일 간 걷기 일수'] - user_data[2], 2)
        if diff_exercise > 0:
            checklist.add_feedback(condition_name, "1주일 간 걷기 일수", "늘려야", abs(diff_exercise))
            print(f"1주일 간 걷기 일수는 **{abs(diff_exercise)}회 더 늘려야** 합니다.")
        elif diff_exercise < 0:
            checklist.add_feedback(condition_name, "1주일 간 걷기 일수", "충분", abs(diff_exercise))
            print(f"1주일 간 걷기 일수는 **충분합니다.**")
        else:
            checklist.add_feedback(condition_name, "1주일 간 걷기 일수", "적정", "적정")
            print("1주일 간 걷기 일수는 **적정입니다.**")
        diff_breakfast = round(target_cluster_means['1주일 간 아침식사 빈도'] - user_data[3], 2)
        if diff_breakfast > 0:
            checklist.add_feedback(condition_name, "1주일 간 아침식사 빈도", "늘려야", abs(diff_breakfast))
            print(f"1주일 간 아침식사 빈도는 **{abs(diff_breakfast)}회 더 늘려야** 합니다.")
        elif diff_breakfast < 0:
            checklist.add_feedback(condition_name, "1주일 간 아침식사 빈도", "충분", abs(diff_breakfast))
            print(f"1주일 간 아침식사 빈도는 **충분합니다.**")
        else:
            checklist.add_feedback(condition_name, "1주일 간 아침식사 빈도", "적정", "적정")
            print("1주일 간 아침식사 빈도는 **적정입니다.**")

    # 고혈압 분석
    he_hp2 = determine_he_hp2(sbp, dbp)
    he_hp2_labels = {1: "정상", 2: "주의 혈압", 3: "고혈압 전단계", 4: "고혈압"}
    if not pd.isna(he_hp2):
        analyze_condition('HE_HP2', he_hp2, he_hp2_labels, [weekly_alcohol, daily_smoking, weekly_exercise, daily_breakfast])
    else:
        print(f"\n혈압 수치 ({sbp}/{dbp})가 유효하지 않아 고혈압 분류를 할 수 없습니다.")

    # 당뇨 분석
    he_dm_hba1c2 = determine_he_dm_hba1c2(glu)
    he_dm_hba1c2_labels = {1: "정상", 2: "당뇨 전단계", 3: "당뇨"}
    if not pd.isna(he_dm_hba1c2):
        analyze_condition('HE_DM_HbA1c2', he_dm_hba1c2, he_dm_hba1c2_labels, [weekly_alcohol, daily_smoking, weekly_exercise, daily_breakfast])
    else:
        print(f"\n공복혈당 수치 ({glu})가 유효하지 않아 당뇨 분류를 할 수 없습니다.")

    # 비만 분석
    he_obe2 = determine_he_obe2(bmi)
    he_obe2_labels = {1: "저체중", 2: "정상", 3: "과체중", 4: "비만", 5: "고도비만", 6: "초고도비만"}
    if not pd.isna(he_obe2):
        analyze_condition('HE_obe2', he_obe2, he_obe2_labels, [weekly_alcohol, daily_smoking, weekly_exercise, daily_breakfast])
    else:
        print(f"\nBMI 수치 ({bmi})가 유효하지 않아 비만 분류를 할 수 없습니다.")

    # 체크리스트 출력 및 저장
    checklist.display_checklist()
    checklist.save_checklist("health_checklist.json")

    # 예시 체크
    checklist.check_item("고혈압", "1주일 간 음주 빈도")
    checklist.check_item("당뇨", "1주일 간 걷기 일수")
    checklist.check_item("비만", "하루 평균 흡연량")
    print("\n=== 체크 후 건강 체크리스트 상태 ===")
    checklist.display_checklist()

    return checklist.get_checklist_status()

# 서비스 실행
if __name__ == "__main__":
    checklist_data = integrated_health_service()
    print("\n=== JSON 형식 체크리스트 데이터 (어플 연동용) ===")
    print(json.dumps(checklist_data, ensure_ascii=False, indent=4))

C:\Users\asia\AppData\Local\Temp\ipykernel_12972\1783291180.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['BE3_31'] = df['BE3_31'].astype(str).str.strip().replace({
C:\Users\asia\AppData\Local\Temp\ipykernel_12972\1783291180.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['L_BR_FQ'] = df['L_BR_FQ'].astype(str).str.strip().replace({
C:\Users\asia\AppData\Local\Temp\ipykernel_12972\1783291180.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the


---
## ✅ 고혈압 건강 목표 - 더 건강한 그룹과 나 비교하기

---
## ✅ 사용자 생활습관 vs 비교 군집 평균 상세:
| 항목                   |   홍길동님 수치 |   비교 군집 수치 |
|:-----------------------|----------------:|-----------------:|
| 1주일 간 음주 빈도     |            3.00 |             0.46 |
| 하루 평균 흡연량       |           10.00 |             0.33 |
| 1주일 간 걷기 일수     |            2.00 |             6.29 |
| 1주일 간 아침식사 빈도 |            2.00 |             5.84 |

**홍길동**님은 **고혈압**에 속합니다.
1주일 간 음주 빈도는 **2.54회 줄여야** 합니다.
하루 평균 흡연량은 **9.67개비 줄여야** 합니다.
1주일 간 걷기 일수는 **4.29회 더 늘려야** 합니다.
1주일 간 아침식사 빈도는 **3.84회 더 늘려야** 합니다.

---
## ✅ 당뇨 건강 목표 - 더 건강한 그룹과 나 비교하기

---
## ✅ 사용자 생활습관 vs 비교 군집 평균 상세:
| 항목                   |   홍길동님 수치 |   비교 군집 수치 |
|:-----------------------|----------------:|-----------------:|
| 1주일 간 음주 빈도     |            3.00 |             1.80 |
| 하루 평균 흡연량       |           10.00 |            21.49 |
| 1주일 간 걷기 일수     |            2.00 |             3.80 |
| 1주일 간 아침식사 빈도 |            2.00 |             5.88 |

**홍길동**님은 **당